In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import from_unixtime,col


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1574832680492_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
glueContext = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
db_name = 'db'
tbl_name = 'table1'
dynamic_frame_a = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = tbl_name)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
dynamic_frame_a.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- processing_time: long
|-- record_timestamp: long
|-- .... 
|-- partition_3: string

In [5]:
dynamic_frame_a.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10000

In [6]:
df_a = dynamic_frame_a.toDF()
df_a.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+--------+---+-----------------+
|   timestamp_na| value_a| value_b|...|           format|
+---------------+--------+--------+---+-----------------+
|  1556065073077|   155.0|    null|...|compacted.parquet|
|  1556657541428|   155.0|    null|...|compacted.parquet|
|  1556065073077|   155.0|    null|...|compacted.parquet|
|  1556657541428|   155.0|    null|...|compacted.parquet|
|  1556065073077|   155.0|    null|...|compacted.parquet|
+---------------+--------+--------+---+-----------------+
only showing top 5 rows

In [7]:
df_a.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- processing_time: long (nullable = true)
 |-- record_timestamp: long (nullable = true)
 |-- ainat160_value_d: double (nullable = true)
 |-- ainat161_value_d: null (nullable = true)
 |-- ainat162_value_d: null (nullable = true)
 |-- ainbsc001_status1_value_d: double (nullable = true)
 |-- ainbsc001_status2_value_d: double (nullable = true)
 |-- ainct430_value_d: double (nullable = true)
 |-- ainct435_value_d: null (nullable = true)
 |-- ainct450_value_d: double (nullable = true)
 |-- ainextpowmeas_value_d: double (nullable = true)
 |-- ainfan150control_value_d: double (nullable = true)
 |-- ainft000hrm_value_d: double (nullable = true)
 |-- ainft012_value_d: double (nullable = true)
 |-- ainft140_value_d: double (nullable = true)
 |-- ainft470_value_d: double (nullable = true)
 |-- ainft490hrm_value_d: double (nullable = true)
 |-- ainft491_value_d: double (nullable = true)
 |-- ainft800hrm_value_d: double (nullable = true)
 |-- ainft801_value_d: double (nullable = true)
 |-- a

In [8]:
df_a.describe(['value_a']).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|  ainat160_value_d|
+-------+------------------+
|  count|             10000|
|   mean| 6.024374980056286|
| stddev|3.6867223613722095|
|    min|0.4459261894226074|
|    max| 12.39466381072998|
+-------+------------------+

In [9]:
#When you use 'summary' instead of 'describe' it shows more detail information about data like quantiles
df_a.select('value_a').summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|  ainat160_value_d|
+-------+------------------+
|  count|             10000|
|   mean| 6.024374980056286|
| stddev|3.6867223613722095|
|    min|0.4459261894226074|
|    25%|1.0007023811340332|
|    50%| 6.811797618865967|
|    75%| 8.946630477905273|
|    max| 12.39466381072998|
+-------+------------------+

In [11]:
df_a.select('timestamp_na').head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(processing_time=1556065073077), Row(processing_time=1556657541428), Row(processing_time=1556065073077)]

In [12]:
# Data Cleansing: When the value of timestamp_ap is like '1548883419015', convert it to normal timestamp formation.
df_a.select(from_unixtime(col('timestamp_na')/1000)).head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(from_unixtime((processing_time / 1000), yyyy-MM-dd HH:mm:ss)='2019-04-24 00:17:53')

In [13]:
df_a.select('timestamp_ap').head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(record_timestamp=1556064777170)

In [14]:
df_a.select(from_unixtime(col('timestamp_ap')/1000)).head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(from_unixtime((record_timestamp / 1000), yyyy-MM-dd HH:mm:ss)='2019-04-24 00:12:57')

In [15]:
df_a_tmp = df_a\
    .withColumn('converted_timestamp_na' , from_unixtime(col('timestamp_na')/1000))\
    .withColumn('converted_timestamp_ap', from_unixtime(col('timestamp_ap')/1000))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_a_tmp.select('converted_timestamp_na','converted_timestamp_ap').show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+--------------------------+
|converted_processing_time|converted_record_timestamp|
+-------------------------+--------------------------+
|      2019-04-24 00:17:53|       2019-04-24 00:12:57|
|      2019-04-30 20:52:21|       2019-04-30 20:48:33|
|      2019-04-24 00:17:53|       2019-04-24 00:12:58|
|      2019-04-30 20:52:21|       2019-04-30 20:48:34|
|      2019-04-24 00:17:53|       2019-04-24 00:12:59|
+-------------------------+--------------------------+
only showing top 5 rows

In [17]:
dyf_cleasing = DynamicFrame.fromDF(df_a_tmp,glueContext,'test')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
datasink0 = glueContext.write_dynamic_frame.from_options(\
                                                         frame = dyf_cleasing,\
                                                         connection_type = "s3",\
                                                         connection_options = {"path": "s3a://datalake/etl_sample/"},\
                                                         format = "parquet", transformation_ctx = "datasink0")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…